In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import folium
from IPython.display import IFrame


In [2]:
import math

def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)


In [3]:
# Put here the coordinates of the centre of the area of analysis
# For Sydney: 
c_point = (-33.882882592547034, 151.2063073174587)
# For Zurich:
#c_point = (47.372133, 8.516359)

#### Import the dataframe with the POIs, including the calculated POI density
Then, import the dataframe with the nodes

In [6]:
# Import the dataframe with the POIs
poi_df = pd.read_csv(r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\POI_df.csv')
# Import the dataframe with the nodes
node_csv = pd.read_csv(r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\Urban network analysis_nodes.csv')
node_csv

,Unnamed: 0,y,x,osmid,ref,highway,geometry
0,4032839686,-33.884170,151.206674,4032839686,NaN,NaN,POINT (151.2066739 -33.8841699)
1,5603655691,-33.889497,151.208640,5603655691,NaN,NaN,POINT (151.2086402 -33.8894974)
2,5603655694,-33.889278,151.208666,5603655694,NaN,NaN,POINT (151.2086659 -33.8892777)
3,5603655695,-33.889505,151.208731,5603655695,NaN,NaN,POINT (151.2087313 -33.8895051)
4,5603655696,-33.889432,151.208740,5603655696,NaN,NaN,POINT (151.2087396 -33.8894318)
...,...,...,...,...,...,...,...
2255,1846288364,-33.889820,151.199394,1846288364,NaN,NaN,POINT (151.1993935 -33.8898197)
2256,5243953133,-33.885422,151.206902,5243953133,NaN,NaN,POINT (151.2069021 -33.8854224)
2257,1846288366,-33.890985,151.198651,1846288366,NaN,NaN,POINT (151.1986507 -33.8909847)
2258,5851340786,-33.876857,151.205297,5851340786,NaN,NaN,POINT (151.2052967 -33.8768565)


#### Create arrays to store information regarding the distance from the closest poi and its POI density

In [7]:
Closest_Poi_Density_array = np.zeros(len(node_csv))
Closest_Poi_Distance_array = np.zeros(len(node_csv))
Closest_Poi_Index_array = np.zeros(len(node_csv))

#### Construct a kd-tree for the poi data

In [8]:
places = []
for index, row in poi_df.iterrows():
    coordinates = [row['Latitude'], row['Longitude']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)
tree = spatial.KDTree(places)

#### Query the kd-tree that has the spatial proximity information for the poi data
 For each street network node, find the closest POI and store the distance from it,and the POI density

In [9]:
node_csv['Closest Poi Density']=0
node_csv['Closest Poi Distance']=0
node_csv['Closest Poi Index']=0

for item in node_csv.index:
    this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])
    closest = tree.query([cartesian(this_pt[0],this_pt[1])], p = 2)
    closest_distance, closest_poi_index = closest[0][0]*1000, closest[1][0]
    closest_poi = (poi_df.loc[poi_df.index[closest_poi_index], 'Latitude'],poi_df.loc[poi_df.index[closest_poi_index], 'Longitude'])
    closest_poi_density = poi_df.loc[poi_df.index[closest_poi_index], 'Density']
    
    node_density = closest_poi_density
    
    Closest_Poi_Density_array[item] = node_density
    Closest_Poi_Distance_array[item] = closest_distance
    Closest_Poi_Index_array[item] = closest_poi_index


#### Transform the POI density so that it takes into account the distance from the closest POI 
The logic is that in some suburban areas the closest POI is far away

So the nodes that are far away from the closest POI get a reduced POI density value instead of the initial one


In [10]:
node_csv['Closest Poi Density']=Closest_Poi_Density_array 
node_csv['Closest Poi Distance']=Closest_Poi_Distance_array
node_csv['Closest Poi Index']=Closest_Poi_Index_array
node_csv['Closest Poi Density_with distance decay']=Closest_Poi_Distance_array

new_dens =(node_csv['Closest Poi Density']/np.log1p(node_csv['Closest Poi Distance']).apply(np.ceil)).apply(np.ceil).values**2
node_csv['Closest Poi Density_with distance decay']=new_dens
#this results in some cases where the new density is larger than the old one. So we use the following to replace it
max_density = node_csv['Closest Poi Density']
node_csv.loc[node_csv[node_csv['Closest Poi Density_with distance decay']>max_density].index, 'Closest Poi Density_with distance decay']=node_csv.loc[node_csv[node_csv['Closest Poi Density_with distance decay']>max_density].index, 'Closest Poi Density']
node_csv

,Unnamed: 0,y,x,osmid,ref,highway,geometry,Closest Poi Density,Closest Poi Distance,Closest Poi Index,Closest Poi Density_with distance decay
0,4032839686,-33.884170,151.206674,4032839686,NaN,NaN,POINT (151.2066739 -33.8841699),23.0,51.395745,1008.0,23.0
1,5603655691,-33.889497,151.208640,5603655691,NaN,NaN,POINT (151.2086402 -33.8894974),19.0,55.902375,1060.0,16.0
2,5603655694,-33.889278,151.208666,5603655694,NaN,NaN,POINT (151.2086659 -33.8892777),3.0,49.442273,1143.0,1.0
3,5603655695,-33.889505,151.208731,5603655695,NaN,NaN,POINT (151.2087313 -33.8895051),16.0,60.820926,1039.0,16.0
4,5603655696,-33.889432,151.208740,5603655696,NaN,NaN,POINT (151.2087396 -33.8894318),3.0,60.132376,1143.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2255,1846288364,-33.889820,151.199394,1846288364,NaN,NaN,POINT (151.1993935 -33.8898197),11.0,104.125530,146.0,9.0
2256,5243953133,-33.885422,151.206902,5243953133,NaN,NaN,POINT (151.2069021 -33.8854224),26.0,13.717985,276.0,26.0
2257,1846288366,-33.890985,151.198651,1846288366,NaN,NaN,POINT (151.1986507 -33.8909847),14.0,33.273068,1233.0,14.0
2258,5851340786,-33.876857,151.205297,5851340786,NaN,NaN,POINT (151.2052967 -33.8768565),17.0,40.630825,224.0,17.0


#### Visualise POI density

In [11]:
osm_and_poi = folium.Map(location = [c_point[0],c_point[1]], zoom_start = 20, tiles='CartoDB dark_matter' ) 
osm_and_poi.save("POI density projected on street network nodes.html" )



for item in node_csv.index:

    this_pt = (node_csv.loc[item, 'y'],node_csv.loc[item, 'x'])
    node_density = node_csv.loc[item,'Closest Poi Density_with distance decay']

    this_color_poi='green'
    this_color_node='green'
    
    # colour scheme for visualisation
    if node_density>=50:
        this_color_node='darkpurple'
    elif (node_density>40) & (node_density<=50):
        this_color_node='purple'
    elif (node_density>30) & (node_density<=40):
        this_color_node='darkred'
    elif (node_density>20) & (node_density<=30):
        this_color_node='red'
    elif (node_density>10) & (node_density<=20):
        this_color_node = 'orange'
    elif (node_density>5) & (node_density<=10):
        this_color_node = 'beige'
    else:
        this_color_node = 'green'
    
    folium.CircleMarker([this_pt[0],this_pt[1]], color=this_color_node, fill=True, radius=0.5, opacity=0.5,
                   popup = 'node ' + str(item)).add_to(osm_and_poi)                                          

osm_and_poi.save("POI density projected on street network nodes.html" )

IFrame(src='./POI density projected on street network nodes.html', width=1500, height=1500)


In [12]:
# Replace the old 'Urban network analysis_nodes' csv with the modified one, which now has the POI density projected on the street network nodes
node_csv.to_csv(r'C:\Users\demdr\Desktop\Testing the thesis functions\Project data\Spatial database\Urban network analysis_nodes.csv')